In [1]:
# declare a list tasks whose products you want to use as inputs
upstream = ['fetch_securities']


In [2]:
# Parameters
upstream = {
    "fetch_securities": {
        "nb": "/Users/aiujdm2/market_watch/output/notebooks/fetch_securities.ipynb",
        "data": "/Users/aiujdm2/market_watch/output/data/raw/securities.csv",
    }
}
product = {
    "nb": "/Users/aiujdm2/market_watch/output/notebooks/normalize_security_names.ipynb",
    "data": "/Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv",
}


In [3]:
import ast
import pandas as pd
import numpy as np
import json
from pathlib import Path
from src.utils import preprocess_text

In [4]:
securities_file_path = upstream['fetch_securities']['data']
securities_df = pd.read_csv(securities_file_path, index_col=0)

In [5]:
securities_df.drop_duplicates(subset=['cik'], inplace=True)

In [6]:
securities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9213 entries, 0 to 9212
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cik                    9213 non-null   int64  
 1   ticker                 9213 non-null   object 
 2   exchanges              9213 non-null   object 
 3   ein                    9203 non-null   float64
 4   full_name              9213 non-null   object 
 5   former_names           9213 non-null   object 
 6   short_name             499 non-null    object 
 7   gics_sector            499 non-null    object 
 8   gics_sub_industry      499 non-null    object 
 9   sic                    8428 non-null   float64
 10  sic_description        8414 non-null   object 
 11  headquarters_location  499 non-null    object 
dtypes: float64(2), int64(1), object(9)
memory usage: 935.7+ KB


In [7]:
securities_df.former_names.replace('[]', np.nan, inplace=True)
securities_df[securities_df.former_names.isnull()]

,cik,ticker,exchanges,ein,full_name,former_names,short_name,gics_sector,gics_sub_industry,sic,sic_description,headquarters_location
1,947559,FBMS,['Nasdaq'],640862173.0,FIRST BANCSHARES INC /MS/,NaN,NaN,NaN,NaN,6021.0,National Commercial Banks,NaN
4,1515317,MAGE,['OTC'],273566922.0,MAGELLAN GOLD Corp,NaN,NaN,NaN,NaN,1000.0,Metal Mining,NaN
7,1294325,CLPBY,"['OTC', 'OTC']",0.0,COLOPLAST A/S,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1844336,FTVI,"['Nasdaq', 'Nasdaq', 'Nasdaq']",853742586.0,FINTECH ACQUISITION CORP VI,NaN,NaN,NaN,NaN,6770.0,Blank Checks,NaN
10,1734262,CTK,['NYSE'],0.0,CooTek(Cayman)Inc.,NaN,NaN,NaN,NaN,7374.0,Services-Computer Processing & Data Preparation,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
9203,8858,AVT,['Nasdaq'],111890605.0,AVNET INC,NaN,NaN,NaN,NaN,5065.0,"Wholesale-Electronic Parts & Equipment, NEC",NaN
9204,8063,ATRO,"['Nasdaq', 'OTC']",160959303.0,ASTRONICS CORP,NaN,NaN,NaN,NaN,3728.0,"Aircraft Parts & Auxiliary Equipment, NEC",NaN
9207,1692063,SNDR,['NYSE'],391258315.0,"Schneider National, Inc.",NaN,NaN,NaN,NaN,4213.0,Trucking (No Local),NaN
9210,1874981,USAX,[''],863599901.0,USA Acquisition Corp.,NaN,NaN,NaN,NaN,6770.0,Blank Checks,NaN


In [8]:
def nomalize_names(securities_df):
    def extract_name(string):
        if len(str(string).strip()) > 5:
            string = ast.literal_eval(string)
            string = json.dumps(string)
            lst = json.loads(string)
            name = lst[0]["name"]
            return name
        else:
            return np.nan
            
    securities_df['former_name'] = securities_df.former_names.apply(extract_name)
    securities_df = securities_df.replace(to_replace=["/[A-Za-z]+/?"], value=[''], regex=True)
    securities_df = securities_df.replace(to_replace=[r'\\DE\\'], value=[''], regex=True)
    
    not_null_cond = ~securities_df['former_name'].isnull()
    
    securities_df.loc[not_null_cond,'former_name'] = preprocess_text(securities_df.loc[not_null_cond,'former_name'])
    
    securities_df['full_name'] = preprocess_text(securities_df.full_name)
    securities_df.replace('[]', np.nan, inplace=True)

    columns_to_select = ['cik','ticker','full_name', 'former_name']
    
    return securities_df[columns_to_select]

In [9]:
securities_df = nomalize_names(securities_df)

In [10]:
output_file_path = product['data']
Path(output_file_path).parent.mkdir(exist_ok=True, parents=True)
securities_df.to_csv(output_file_path)
print(f"Saved file {output_file_path}")

Saved file /Users/aiujdm2/market_watch/output/data/interim/normalized_security_names.csv
